In [ ]:
import cv2
import numpy as np
import pandas as pd
import random
import glob
import csv
import keras
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def tiles_images(img, pieces):
  img_width = img_height = img.shape[1] // pieces

  tiles_img = [img[x:x+img_height,y:y+img_width] 
             for x in range(0,img.shape[0], img_height) 
             for y in range(0,img.shape[1],img_width)
            ]

  new_tiles = []

  for img in tiles_img:
    if img.shape[0] == img_width and  img.shape[1] == img_height:
      new_tiles.append(img)

  return new_tiles

In [ ]:
def group_images(images, shuffle_arr, pieces=4):
  new_img = []

  shuffle_tiles =  [images[i] for i in shuffle_arr]

  for i in range(pieces):
    new_img.append(np.concatenate(shuffle_tiles[i*pieces:(i+1)*pieces], axis=1))
    
  new_img = np.concatenate(new_img, axis=0)
  return new_img

In [ ]:
image = cv2.imread('../input/jigsaw-puzzle/Gravity Falls/correct/0.png') 
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.subplots(figsize=(4,4))
plt.imshow(image)

In [ ]:
pieces = 4     # number pieces in one row/column 
tiles = tiles_images(image, pieces)

fig, ax = plt.subplots(pieces, pieces, figsize=(4, 4))

for i, idx in enumerate(range(pieces**2)):
  plt.subplot(pieces, pieces, i+1)
  plt.imshow(tiles[idx])
  plt.axis("off")

Shuffle puzzles ang get Y vector

In [ ]:
shuffle_arr = np.random.permutation(len(tiles))

In [ ]:
shuffle_arr

In [ ]:
fig, ax = plt.subplots(pieces, pieces, figsize=(4, 4))

for i, idx in enumerate(shuffle_arr):
  plt.subplot(pieces, pieces, i+1)
  plt.imshow(tiles[idx])
  plt.axis("off")

In [ ]:
shuffle_img = group_images(tiles, shuffle_arr)

plt.subplots(figsize=(4, 4))
plt.imshow(shuffle_img)

## Create dataset

In [ ]:
correct_image_path = "../input/jigsaw-puzzle/Gravity Falls/correct/*"
correct_image = [cv2.imread(file) for file in glob.glob(correct_image_path)]

In [ ]:
X, Y = [], []
pieces = 8   # number pieces in one row/column

data_type = 'train'            #change for test, train and validation data
X_path = "puzzle_8x8/"+data_type+"/"

!mkdir puzzle_8x8             #create directory
!mkdir {X_path}

n = 10 if data_type=='train' else 1        #for train dataset create 10x images

for j in range(n):
  for i, img in enumerate(correct_image):
    tiles = tiles_images(img, pieces)

    array = np.random.permutation(len(tiles))
    Y.append(array)

    group_img = group_images(tiles, array, pieces)
    X.append(group_img)

    path = f"{X_path}{i+j*110}.jpg"
    cv2.imwrite(path, group_img)
    
with open("puzzle_8x8/"+data_type+'.csv', 'w') as file:
  writer = csv.writer(file)
  head = [i for i in range(pieces**2)]
  writer.writerow(head)
  writer.writerows(Y)